## FISRT NOTEBOOK -- TEAM NOVICE

## 1. INTRODUCTION

The objective of the AI4EO HYPERVIEW challenge is to predict agriculturally relevant soil parameters (K, Mg, P2O5, pH) from airborne hyperspectral images. We present a hybrid model fusing Random Forest and K-nearest neighbor regressors that exploit the average spectral
reflectance, as well as derived features such as gradients, wavelet coefficients, and Fourier transforms. The solution is computationally lightweight and improves upon the challenge baseline by 21%.

## 2. SETUP

In the following cell all modules are imported, that are needed in this notebook.

In [ ]:
!pip install PyWavelets

In [ ]:
# GENERAL UTILITIES
import os
from glob import glob
import pandas as pd
from  tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

# MODEL DEVELOPMENT DEPENDENCIES
import numpy as np
import pywt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing


In [ ]:
# SOME CONSTANTS UTILIZED IN THE NOTEBOOK

DEBUG = False
AUGMENT_CONSTANT_RF=1
AUGMENT_CONSTANT_KNN=6
LABEL_NAMES = ["P2O5", "K", "Mg", "pH"]
LABEL_MAXS = np.array([325.0, 625.0, 400.0, 7.8])
#Y_BASE_FACT = np.array([121764.2 / 1731.0, 394876.1 / 1731.0, 275875.1 / 1731.0, 11747.67 / 1731.0]) / LABEL_MAXS

COL_IX = [0, 1, 2, 3]


## 3. DATA LOADING

In addition to loading the data in a standard way provided by original notebook, we make  pre-augmentation by loading randomly cropped 11x11 size fields, because the main performance loss is coming from this part of the data.

In [ ]:

def load_data(directory: str, gt_file_path: str, is_train=True, augment_constant: int = 0):
    """Load each cube, reduce its dimensionality and append to array.

    Args:
        directory (str): Directory to either train or test set
        gt_file_path (str): File path for the ground truth labels (expected CVS file)
        is_train (boolean): Binary flag for setting loader for Train (TRUE) or Test (FALSE)
        augment_constant (int): number of augmentation steps to randomly crop from the larger agricultural fields
    Returns:
        [type]: Tuple of lists composed of raw field (data , mask) pairs,
                and if exists: (augmented data, augmented mask) pairs, and ground truth labels
    """

    datalist = []
    masklist = []
    aug_datalist = []
    aug_masklist = []
    aug_labellist = []

    if is_train:
        labels = load_gt(gt_file_path)

    all_files = np.array(
        sorted(
            glob(os.path.join(directory, "*.npz")),
            key=lambda x: int(os.path.basename(x).replace(".npz", "")),
        )
    )

    if DEBUG:
        all_files = all_files[:100]
        if is_train:
            labels = labels[:100]

    for idx, file_name in tqdm(enumerate(all_files),total=len(all_files), desc="Loading {} data .."
                               .format("training" if is_train else "test")):
       # We load the data into memory as provided in the example notebook of the challenge
        with np.load(file_name) as npz:
            mask = npz["mask"]
            data = npz["data"]
            datalist.append(data)
            masklist.append(mask)

    # for training data we make pre-augmentation by adding some randomly cropped samples
    if is_train:
        for i in range(augment_constant):
            for idx, file_name in tqdm(enumerate(all_files),total=len(all_files), desc="Loading augmentation {} ..".format(i+1)):
                # print(file_name)
                with np.load(file_name) as npz:
                    flag = True
                    mask = npz["mask"]
                    data = npz["data"]
                    ma = np.max(data, keepdims=True)
                    sh = data.shape[1:]
                    for i in range(10):
                        # Repeating 11x11 cropping 10 times does not mean we use all croppings:
                        # as seen in the Flag=False below at the end of the loop,
                        # when we reach at the good crop (not coinciding to the masked area) we stop searching

                        # Randomly cropping the fields with 11x11 size,
                        # and adding some noise to the cropped samples
                        edge = 11
                        x = np.random.randint(sh[0] + 1 - edge)
                        y = np.random.randint(sh[1] + 1 - edge)

                        # get crops having meaningful pixels, not zeros
                        if np.sum(mask[0, x : (x + edge), y : (y + edge)]) > 120:
                            aug_data = (data[:, x : (x + edge), y : (y + edge)]
                                        + np.random.uniform(-0.01, 0.01, (150, edge, edge)) * ma)
                            aug_mask = mask[:, x : (x + edge), y : (y + edge)] | np.random.randint(0, 1, (150, edge, edge))

                            flag = False #break the loop when you have a meaningful crop
                            break

                    # After having  11x11 croped sample, get another crop considering
                    # the minimum edge length: (min_edge,min_edge)
                    if flag:
                        max_edge = np.max(sh)
                        min_edge = np.min(sh)  # AUGMENT BY SHAPE
                        edge = min_edge  # np.random.randint(16, min_edge)
                        x = np.random.randint(sh[0] + 1 - edge)
                        y = np.random.randint(sh[1] + 1 - edge)
                        aug_data = (data[:, x : (x + edge), y : (y + edge)]
                                    + np.random.uniform(-0.001, 0.001, (150, edge, edge)) * ma)
                        aug_mask = mask[:, x : (x + edge), y : (y + edge)] | np.random.randint(0, 1, (150, edge, edge))

                    aug_datalist.append(aug_data)
                    aug_masklist.append(aug_mask)
                    aug_labellist.append(
                        labels[idx, :]
                        + labels[idx, :] * np.random.uniform(-0.001, 0.001, 4)
                    )

    # do pre-augmentation only for training data
    if is_train:
        return (datalist,
                masklist,
                labels,
                aug_datalist,
                aug_masklist,
                np.array(aug_labellist))
    else:
        return datalist, masklist


def load_gt(file_path: str):
    """Load labels for train set from the ground truth file.
    Args:
        file_path (str): Path to the ground truth .csv file.
    Returns:
        [type]: 2D numpy array with soil properties levels
    """
    gt_file = pd.read_csv(file_path)
    labels = gt_file[["P", "K", "Mg", "pH"]].values / LABEL_MAXS  # normalize ground-truth between 0-1

    return labels

In [ ]:
# Please be sure that the directory and file locations are given correctly in your own system
train_data_dir = "train_data"
test_data_dir = "test_data"
gt_data_path = "train_gt.csv"

# Loading training raw data
X_train, M_train, y_train, X_aug_train, M_aug_train, y_aug_train = load_data(train_data_dir,
                                                                             gt_data_path,
                                                                             is_train=True,
                                                                             augment_constant=AUGMENT_CONSTANT_KNN)
# Loading test raw data
X_test, M_test = load_data(test_data_dir,
                           gt_file_path=None,
                           is_train=False)

print(f"Train data size: {len(X_train)}")
print(f"Train aug data size: {len(X_aug_train)}")
print(f"Test data size: {len(X_test)}")

Loading training data ..:   0%|          | 0/1732 [00:00<?, ?it/s]

Loading augmentation 1 ..:   0%|          | 0/1732 [00:00<?, ?it/s]

Loading augmentation 2 ..:   0%|          | 0/1732 [00:00<?, ?it/s]

Loading augmentation 3 ..:   0%|          | 0/1732 [00:00<?, ?it/s]

Loading augmentation 4 ..:   0%|          | 0/1732 [00:00<?, ?it/s]

Loading augmentation 5 ..:   0%|          | 0/1732 [00:00<?, ?it/s]

Loading augmentation 6 ..:   0%|          | 0/1732 [00:00<?, ?it/s]

Loading test data ..:   0%|          | 0/1154 [00:00<?, ?it/s]

Train data size: 1732
Train aug data size: 10392
Test data size: 1154


## 4. PREPROCESSING THE LOADED DATA

We have multiple preprocessing techniques to extract features before training in the function below:

* Wavelet Transform
* Fast Fourier Transform
* Singular Value Decomposition
* First, Second and Third Derivative of the Original Data
* and some combinations of the methods above

(For more information about the feature selection, you can look at the attached pre-print paper for ICIP2022 Conference)
    
    


In [ ]:
def preprocess(data_list, mask_list, is_for_KNN=False):
    """Extract high-level features from the raw field data.

    Args:
        data_list: Directory to either train or test set
        mask_list: File path for the ground truth labels (expected CVS file)
        is_for_KNN: Binary flag for determining if the features are generated for KNN (TRUE) or Random Forest (FALSE)
    Returns:
        [type]: Tuple of lists composed of (features , field size) pairs for each field,
                where field size will be used performance analysis.
    """

    def _shape_pad(data):
        # This sub-function makes padding to have square fields sizes.
        # Not mandatory but eliminates the risk of calculation error in singular value decomposition,
        # padding by warping also improves the performance slightly.
        max_edge = np.max(image.shape[1:])
        shape = (max_edge, max_edge)
        padded = np.pad(data,((0, 0), (0, (shape[0] - data.shape[1])), (0, (shape[1] - data.shape[2]))),"wrap")
        return padded

    filtering = SpectralCurveFiltering()
    w1 = pywt.Wavelet("sym3")
    w2 = pywt.Wavelet("dmey")

    processed_data = []
    average_edge = []

    for idx, (data, mask) in enumerate(
        tqdm(
            zip(data_list, mask_list),
            total=len(data_list),
            position=0,
            leave=True,
            desc="INFO: Preprocessing data ...",
        )
    ):
        data = data / 2210   # max-max=5419 mean-max=2210
        m = 1 - mask.astype(int)
        image = data * m

        average_edge.append((image.shape[1] + image.shape[2]) / 2)
        image = _shape_pad(image)

        s = np.linalg.svd(image, full_matrices=False, compute_uv=False)
        s0 = s[:, 0]
        s1 = s[:, 1]
        s2 = s[:, 2]
        s3 = s[:, 3]
        s4 = s[:, 4]
        dXds1 = s0 / (s1 + np.finfo(float).eps)


        data = np.ma.MaskedArray(data, mask)
        arr = filtering(data)

        cA0, cD0 = pywt.dwt(arr, wavelet=w2, mode="constant")
        cAx, cDx = pywt.dwt(cA0[12:92], wavelet=w2, mode="constant")
        cAy, cDy = pywt.dwt(cAx[15:55], wavelet=w2, mode="constant")
        cAz, cDz = pywt.dwt(cAy[15:35], wavelet=w2, mode="constant")
        cAw2 = np.concatenate((cA0[12:92], cAx[15:55], cAy[15:35], cAz[15:25]), -1)
        cDw2 = np.concatenate((cD0[12:92], cDx[15:55], cDy[15:35], cDz[15:25]), -1)

        cA0, cD0 = pywt.dwt(arr, wavelet=w1, mode="constant")
        cAx, cDx = pywt.dwt(cA0[1:-1], wavelet=w1, mode="constant")
        cAy, cDy = pywt.dwt(cAx[1:-1], wavelet=w1, mode="constant")
        cAz, cDz = pywt.dwt(cAy[1:-1], wavelet=w1, mode="constant")
        cAw1 = np.concatenate((cA0, cAx, cAy, cAz), -1)
        cDw1 = np.concatenate((cD0, cDx, cDy, cDz), -1)

        dXdl = np.gradient(arr, axis=0)
        d2Xdl2 = np.gradient(dXdl, axis=0)
        d3Xdl3 = np.gradient(d2Xdl2, axis=0)


        fft = np.fft.fft(arr)
        real = np.real(fft)
        imag = np.imag(fft)
        ffts = np.fft.fft(s0)
        reals = np.real(ffts)
        imags = np.imag(ffts)

        # The best Feature combination for Random Forest based regression
        out_rf = np.concatenate(
            [
                arr,
                dXdl,
                d2Xdl2,
                d3Xdl3,
                dXds1,
                s0,
                s1,
                s2,
                s3,
                s4,
                real,
                imag,
                reals,
                imags,
                cAw1,
                cAw2,
            ],
            -1,
        )

        # The best Feature combination for KNN based regression
        out_knn = np.concatenate(
            [
                arr,
                dXdl,
                d2Xdl2,
                d3Xdl3,
                s0,
                s1,
                s2,
                s3,
                s4,
                real,
                imag,

            ],
            -1,
        )


        if is_for_KNN:
            processed_data.append(out_knn)
        else:
            processed_data.append(out_rf)

    return np.array(processed_data), np.array(average_edge)



class SpectralCurveFiltering: # Default class provided by the challenge organizers
    """
    Create a histogram (a spectral curve) of a 3D cube, using the merge_function
    to aggregate all pixels within one band. The return array will have
    the shape of [CHANNELS_COUNT]
    """

    def __init__(self, merge_function=np.mean):
        self.merge_function = merge_function

    def __call__(self, sample: np.ndarray):
        return self.merge_function(sample, axis=(1, 2))


We are extractig features for training and test by pre-processing the data:

In [ ]:


# preprocessed data for random forest traninig and testing
X_tr_processed_RF, avg_edge_train = preprocess(X_train, M_train, is_for_KNN=False)
X_aug_processed_RF, avg_edge_train_aug_RF = preprocess(X_aug_train[:len(X_train)*AUGMENT_CONSTANT_RF], M_aug_train[:len(X_train)*AUGMENT_CONSTANT_RF], is_for_KNN=False)
X_te_processed_RF, avg_edge_test = preprocess(X_test, M_test, is_for_KNN=False)

# preprocessed data for KNN traninig and testing
X_tr_processed_KNN, avg_edge_train = preprocess(X_train, M_train, is_for_KNN=True)
X_aug_processed_KNN, avg_edge_train_aug_KNN = preprocess(X_aug_train, M_aug_train,is_for_KNN=True)
X_te_processed_KNN, avg_edge_test = preprocess(X_test, M_test, is_for_KNN=True)


INFO: Preprocessing data ...:   0%|          | 0/1732 [00:00<?, ?it/s]

INFO: Preprocessing data ...:   0%|          | 0/1732 [00:00<?, ?it/s]

INFO: Preprocessing data ...:   0%|          | 0/1154 [00:00<?, ?it/s]

INFO: Preprocessing data ...:   0%|          | 0/1732 [00:00<?, ?it/s]

INFO: Preprocessing data ...:   0%|          | 0/10392 [00:00<?, ?it/s]

INFO: Preprocessing data ...:   0%|          | 0/1154 [00:00<?, ?it/s]

## 5. TRAINING THE MODEL


### 5.A. BASELINE REGRESSOR

In the following cell the baseline regressor, which was given by the challenge is defined. This baseline regressor always predicts the mean for the soil parameters.

In [ ]:
class BaselineRegressor:
    """
    Baseline regressor, which calculates the mean value of the target from the training
    data and returns it for each testing sample.
    """

    def __init__(self):
        self.mean = 0

    def fit(self, X_train: np.ndarray, y_train: np.ndarray):
        self.mean = np.mean(y_train, axis=0)
        self.classes_count = y_train.shape[1]
        return self

    def predict(self, X_test: np.ndarray):
        return np.full((len(X_test), self.classes_count), self.mean)




### 5.B. TRAINING FOR LARGE FIELDS


We are conducting seperate training for small field and large field samples because 11x11 pixel size fields performed poorly in a single training.



#### 5.B.i. CROSS-VALIDATION WITH RANDOM FOREST REGRESSOR


In [ ]:
# Select set of labels

y_train_col = y_train[:, COL_IX]
y_aug_train_col = y_aug_train[:len(y_train_col)*AUGMENT_CONSTANT_RF, COL_IX]

In [ ]:
# 5-fold cross validation for training.

kfold = KFold(shuffle=True, random_state=2022)
kfold.get_n_splits(X_aug_train, y_aug_train_col)

5

We start training with 5-fold cross validation, and preserve each trained model in the list. Later, we will use the list of the models for performance reporting or submission file generation.

In [ ]:

random_forest_models = []
baseline_regressors = []

y_hat_bl = []
y_hat_rf = []
y_v_list_rf = []
edge_v_list_rf = []

for idx, (ix_train, ix_valid) in enumerate(kfold.split(np.arange(0, len(y_train)), avg_edge_train.astype(int))):
    print("CROSS VALIDATION STEP: {}".format(idx))

    # Merge original data with the augmented data on training set
    X_t = np.concatenate((X_tr_processed_RF[ix_train], X_aug_processed_RF[ix_train]), axis=0)
    y_t = np.concatenate((y_train_col[ix_train], y_aug_train_col[ix_train]), axis=0)

    # Filter out Validation set
    X_v = X_tr_processed_RF[ix_valid]
    y_v =y_train_col[ix_valid]
    y_v_list_rf.append(y_v)

    #Field edge sizes will be used for performance analysis later
    edge = avg_edge_train[ix_valid]
    edge_v_list_rf.append(edge)

    # baseline fiting
    baseline = BaselineRegressor()
    baseline.fit(X_t, y_t)
    baseline_regressors.append(baseline)

    # baseline predictions
    y_b = baseline.predict(X_v)
    y_hat_bl.append(y_b)

    # random forest fitting
    model = RandomForestRegressor(n_estimators=1000, n_jobs=-1)
    model.fit(X_t, y_t)
    random_forest_models.append(model)

    # random forest predictions
    y_hat = model.predict(X_v)
    y_hat_rf.append(y_hat)

    print('Prediction score: {}'.format(model.score(X_v, y_v)))



CROSS VALIDATION STEP: 0
Prediction score: 0.12483527979031794
CROSS VALIDATION STEP: 1
Prediction score: 0.16989586170248464
CROSS VALIDATION STEP: 2
Prediction score: 0.15252493004994613
CROSS VALIDATION STEP: 3
Prediction score: 0.15429431279733952
CROSS VALIDATION STEP: 4
Prediction score: 0.1497631232363564


#### 5.B.ii. PERFORMANCE EVALUATION ON VALIDATION SET (FOR LARGE FIELDS)
For the five trained models, the evaluation score is calculated on the validation set.

In [ ]:
scores = 0
for y_hat, y_b, y_v, y_e in zip(y_hat_rf, y_hat_bl, y_v_list_rf, edge_v_list_rf):
    score = 0
    for i in COL_IX:
        print('\n')
        print("Result for parameter: ", LABEL_NAMES[i])
        mse_rf = mean_squared_error(y_v[:, i] * LABEL_MAXS[i], y_hat[:, i] * LABEL_MAXS[i])
        mse_bl = mean_squared_error(y_v[:, i] * LABEL_MAXS[i], y_b[:, i] * LABEL_MAXS[i])
        score += mse_rf / mse_bl
        print(f"Baseline MSE:      {mse_bl:.2f}")
        print(f"Random Forest MSE: {mse_rf:.2f} ({1e2*(mse_rf - mse_bl)/mse_bl:+.2f} %)")


    print('\n')
    print("CV Evaluation score:", score / 4)

    scores += score

print('\n')
print("OVERALL CV EVALUATION SCORE:", scores / 20)



Result for parameter:  P2O5
Baseline MSE:      1078.98
Random Forest MSE: 1001.94 (-7.14 %)


Result for parameter:  K
Baseline MSE:      4633.81
Random Forest MSE: 3715.33 (-19.82 %)


Result for parameter:  Mg
Baseline MSE:      1684.76
Random Forest MSE: 1551.90 (-7.89 %)


Result for parameter:  pH
Baseline MSE:      0.07
Random Forest MSE: 0.06 (-15.65 %)


CV Evaluation score: 0.8737570831202157


Result for parameter:  P2O5
Baseline MSE:      894.49
Random Forest MSE: 789.82 (-11.70 %)


Result for parameter:  K
Baseline MSE:      3913.83
Random Forest MSE: 3149.32 (-19.53 %)


Result for parameter:  Mg
Baseline MSE:      1653.14
Random Forest MSE: 1393.37 (-15.71 %)


Result for parameter:  pH
Baseline MSE:      0.06
Random Forest MSE: 0.05 (-21.37 %)


CV Evaluation score: 0.8292091266275969


Result for parameter:  P2O5
Baseline MSE:      861.49
Random Forest MSE: 811.47 (-5.81 %)


Result for parameter:  K
Baseline MSE:      3673.56
Random Forest MSE: 2982.23 (-18.82 %)




Calculating the validation score for different field sizes, including patches of size 11x11.

In [ ]:
scores = 0
out_table = []


for y_h, y_b, y_v, y_e in zip(y_hat_rf, y_hat_bl, y_v_list_rf, edge_v_list_rf):
    mse_rf = np.square(
        np.subtract(y_h * LABEL_MAXS, y_v * LABEL_MAXS)
    )
    mse_bl = np.square(
        np.subtract(y_v * LABEL_MAXS, y_b * LABEL_MAXS)
    )
    row = np.zeros((len(y_h), 9))
    row[:, 8] = y_e
    row[:, 0:4] = mse_rf
    row[:, 4:8] = mse_bl
    out_table.append(row)

In [ ]:
x = np.concatenate(out_table, 0)
df = pd.DataFrame(
    x,
    columns=["P2O5", "K", "Mg", "pH", "P2O5_avg", "K_avg", "Mg_avg", "pH_avg", "Edge"],
)
df.head(10)
_, bin_edge = np.histogram(df.Edge.values, bins=4)

# We have determined those bin edges after looking at the field size distribuiton
bin_edge = [0, 11, 40, 50, 100, 110, 120, 130, 210]
bin_edge_labels = [
    "0-11",
    "11-40",
    "40-50",
    "50-100",
    "100-110",
    "110-120",
    "120-130",
    "130+",
]
mse_per_edge = np.zeros((len(bin_edge) - 1, 6), dtype=object)

for i in range(1, len(bin_edge)):
    d_temp = df[(df.Edge <= bin_edge[i]) & (df.Edge > bin_edge[i - 1])]
    mse_per_edge[i - 1, 0] = np.mean(d_temp.P2O5.values) / np.mean(
        d_temp.P2O5_avg.values
    )
    mse_per_edge[i - 1, 1] = np.mean(d_temp.K.values) / np.mean(d_temp.K_avg.values)
    mse_per_edge[i - 1, 2] = np.mean(d_temp.Mg.values) / np.mean(d_temp.Mg_avg.values)
    mse_per_edge[i - 1, 3] = np.mean(d_temp.pH.values) / np.mean(d_temp.pH_avg.values)
    mse_per_edge[i - 1, 5] = len(d_temp)
    mse_per_edge[i - 1, 4] = bin_edge_labels[i - 1]


In the following cell, the mean validation score is demonstrated for each soil parameter and field size:

In [ ]:
d_out = pd.DataFrame(mse_per_edge, columns=["P2O5", "K", "Mg", "pH", "Edge", "Len"])
d_out

,P2O5,K,Mg,pH,Edge,Len
0,1.057946,1.015635,1.01712,0.861278,0-11,650
1,0.487523,0.579514,0.531257,0.977514,11-40,94
2,0.722207,0.752374,0.415241,0.787324,40-50,326
3,0.687112,0.655311,0.624791,0.751544,50-100,138
4,0.917833,0.591326,0.388293,0.754186,100-110,113
5,0.887196,0.828525,0.60946,0.751809,110-120,118
6,0.887643,0.763285,0.6605,0.661214,120-130,132
7,0.802656,0.7658,0.844017,0.786326,130+,161


#### 5.B.iii. MAKE PREDICTIONS AND SUBMISSIONS ( FOR THE LARGE FIELDS > 11x11 PIXELS)
Re-train the random forest model with the whole training and augmentation data without splitting them for cross-validation.


In [ ]:

random_forests = []
model = RandomForestRegressor(n_estimators=1000, n_jobs=-1)

X_t = np.concatenate((X_tr_processed_RF, X_aug_processed_RF), axis=0)
y_t = np.concatenate((y_train_col, y_aug_train_col), axis=0)

model.fit(X_t, y_t)

random_forests.append(model)

In the test set, the patches with size 11x11 pixekls (small fields) are at the very beginning, we can therefore exclude them, by simply starting from hard-coded field index `432` or by dynamically filtering out data with smaller field sizes: (```X_te_processed_RF[avg_edge_test>11, :]```). Thus, we make predictions only for large fields with Random Forest Regressor:

In [ ]:
predictions_large = []

#make predictions on multiple models if there are more than 1 model and later mean the results over them
for rf in random_forests:
    pp = rf.predict(X_te_processed_RF[avg_edge_test>11, :]) #X_te_processed_RF[432:, :]
    predictions_large.append(pp)

predictions_large = np.asarray(predictions_large)
predictions_large = np.mean(predictions_large, axis=0)
predictions_large = predictions_large * LABEL_MAXS # predictions were in the range of [0,1] so that we neeed to scale them back.

submission_df = pd.DataFrame(data=predictions_large, columns=["P", "K", "Mg", "pH"])
submission_df.to_csv("submission_large.csv", index_label="sample_index")
submission_df

,P,K,Mg,pH
0,62.073528,269.879470,167.420540,6.828817
1,58.279397,280.515694,175.791144,6.752192
2,77.890719,279.263355,157.539417,6.883160
3,72.298301,271.406046,156.339853,6.838531
4,49.589958,252.538588,188.040150,6.755080
...,...,...,...,...
717,46.936877,169.610192,135.352855,6.628018
718,45.545826,169.880842,135.613945,6.610217
719,76.024305,239.529187,167.534027,6.643992
720,47.691755,182.878088,138.305905,6.612771


### 5.C. TRAINING FOR SMALL FIELDS

We are conducting separate training for small field and large field samples because 11x11 fields performed poorly in a single training. For the small patches we chose a KNN as model, because it showed better performance on this data. We again use a 5-fold cross validation.

#### 5.C.i. MERGE ORIGINAL AND AUGMENTED DATA

In the following cell, we are merging original data with small field size (first 650 samples) and augmented data with randomly cropped fields from the large ones (sample indices larger than 650).

TODO: file index is hard-coded since the field sizes were already checked.
      For a real life scenario, a dynamic ranking of file indices by field size would be better.


In [ ]:

X_tr_processed_normalized_small = np.array(X_tr_processed_KNN[0:650, :], copy=True) #avg_edge_train<=11
X_aug_processed_normalized_large = np.array(X_aug_processed_KNN[650:1732, :], copy=True) #avg_edge_train>11
X_te_processed_normalized = np.array(X_te_processed_KNN, copy=True)

for i in range(1, AUGMENT_CONSTANT_KNN):
    X_aug_processed_normalized_large = np.concatenate(
        [X_aug_processed_normalized_large,
         X_aug_processed_KNN[650 + (i * 1732) : i * 1732 + 1732, :]],0,)

y_train_small = y_train[0:650, :]
avg_edge_train_small= avg_edge_train[0:650]
y_aug_train_large = y_aug_train[650:1732, :]

for i in range(1, AUGMENT_CONSTANT_KNN):
    y_aug_train_large = np.concatenate(
        [y_aug_train_large, y_aug_train[650 + (i * 1732) : (i * 1732) + 1732, :]], 0)

#### 5.C.ii. NORMALIZE DATA
Data normalization improved the performance on KNN but not on Random Forest. That is why we conducted Robust Scaler here only for features to be used in KNN training:

In [ ]:
# Feature normalization
for i in range(int(X_tr_processed_normalized_small.shape[-1] / 150)):
    scaler = preprocessing.RobustScaler()
    scaler.fit(X_tr_processed_normalized_small[:, 150 * i : 150 * i + 150])

    X_tr_processed_normalized_small[:, 150 * i : 150 * i + 150] = scaler.transform(
        X_tr_processed_normalized_small[:, 150 * i : 150 * i + 150])

    X_aug_processed_normalized_large[:, 150 * i : 150 * i + 150] = scaler.transform(
        X_aug_processed_normalized_large[:, 150 * i : 150 * i + 150])

    X_te_processed_normalized[:, 150 * i : 150 * i + 150] = scaler.transform(
        X_te_processed_normalized[:, 150 * i : 150 * i + 150])

#### 5.C.iii. CROSS-VALIDATION WITH KNN REGRESSOR


In [ ]:
# Select set of labels

y_train_small_col = y_train_small[:, COL_IX]
y_aug_train_large_col = y_aug_train_large[:, COL_IX]

In [ ]:
# 5-fold cross validation for training.

kfold = KFold(shuffle=True)
kfold.get_n_splits(X_tr_processed_normalized_small, y_train_small)

5

In [ ]:
#Train the model for small fields only


KNN_models = []
baseline_regressors = []

y_hat_bl = []
y_hat_knn = []
y_v_list_knn = []
edge_v_list_knn = []

for idx, (ix_train, ix_valid) in enumerate(
    kfold.split(np.arange(0, len(y_train_small)))
):
    print("CROSS VALIDATION STEP: ".format(str(idx)))

    X_t = X_tr_processed_normalized_small[ix_train]
    y_t = y_train_small_col[ix_train]

    X_t = np.concatenate((X_tr_processed_normalized_small[ix_train], X_aug_processed_normalized_large), axis=0)
    y_t = np.concatenate((y_train_small_col[ix_train], y_aug_train_large_col), axis=0)

    X_v = X_tr_processed_normalized_small[ix_valid]
    y_v = y_train_small_col[ix_valid]
    y_v_list_knn.append(y_v)


    edge = avg_edge_train_small[ix_valid]
    edge_v_list_knn.append(edge)

    # baseline training
    baseline = BaselineRegressor()
    baseline.fit(X_t, y_t)
    baseline_regressors.append(baseline)

    # baseline predictions
    y_b = baseline.predict(X_v)
    y_hat_bl.append(y_b)


    # KNN training
    model = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=170, weights="distance"))
    model.fit(X_t, y_t)
    KNN_models.append(model)


    # KNN predictions
    y_hat = model.predict(X_v)
    y_hat_knn.append(y_hat)


    print('Prediction score: {}'.format(model.score(X_v, y_v)))



scores = 0
for y_hat, y_b, y_v, y_e in zip(y_hat_knn, y_hat_bl, y_v_list_knn, edge_v_list_knn):
    score = 0
    for i in COL_IX:
        print('\n')
        print("Result for parameter: ", LABEL_NAMES[i])
        mse_knn = mean_squared_error(y_v[:, i] * LABEL_MAXS[i], y_hat[:, i] * LABEL_MAXS[i])
        mse_bl = mean_squared_error(y_v[:, i] * LABEL_MAXS[i], y_b[:, i] * LABEL_MAXS[i])
        score += mse_knn / mse_bl
        print(f"Baseline MSE:      {mse_bl:.2f}")
        print(f"Random Forest MSE: {mse_knn:.2f} ({1e2*(mse_knn - mse_bl)/mse_bl:+.2f} %)")


    print('\n')
    print("CV Evaluation score:", score / 4)

    scores += score

print('\n')
print("OVERALL CV EVALUATION SCORE:", scores / 20)

CROSS VALIDATION STEP: 


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7eff1ca00550>
Traceback (most recent call last):
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _Thread

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7eff1ca00550>
Traceback (most recent call last):
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _Thread

Prediction score: -0.033956877906187344
CROSS VALIDATION STEP: 
Prediction score: -0.013176772046856355


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7eff1ca009d0>
Traceback (most recent call last):
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _Thread

...

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7eff1ca009d0>
Traceback (most recent call last):
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _Thread

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7eff1ca279d0>
Traceback (most recent call last):
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _Thread

Prediction score: -0.00795966083040045
CROSS VALIDATION STEP: 


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7eff1ca00ca0>
Traceback (most recent call last):
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _Thread

Prediction score: -0.009362415934441315


Result for parameter:  P2O5
Baseline MSE:      1398.82
Random Forest MSE: 1454.08 (+3.95 %)


Result for parameter:  K
Baseline MSE:      2001.36
Random Forest MSE: 1833.27 (-8.40 %)


Result for parameter:  Mg
Baseline MSE:      2694.59
Random Forest MSE: 2646.68 (-1.78 %)


Result for parameter:  pH
Baseline MSE:      0.08
Random Forest MSE: 0.06 (-23.29 %)


CV Evaluation score: 0.926209345948315


Result for parameter:  P2O5
Baseline MSE:      774.25
Random Forest MSE: 785.49 (+1.45 %)


Result for parameter:  K
Baseline MSE:      2773.96
Random Forest MSE: 2667.31 (-3.84 %)


Result for parameter:  Mg
Baseline MSE:      2767.05
Random Forest MSE: 2732.16 (-1.26 %)


Result for parameter:  pH
Baseline MSE:      0.08
Random Forest MSE: 0.05 (-37.96 %)


CV Evaluation score: 0.8959745664719774


Result for parameter:  P2O5
Baseline MSE:      1506.73
Random Forest MSE: 1526.31 (+1.30 %)


Result for parameter:  K
Baseline MSE:      2282.00
Ran

#### 5.C.iv. PERFORMANCE EVALUATION ON VALIDATION SET (FOR SMALL FIELDS)


In [ ]:
scores = 0
out_table = []

for y_h, y_b, y_v, y_e in zip(y_hat_knn, y_hat_bl, y_v_list_knn, edge_v_list_knn):
    mse_knn = np.square(np.subtract(y_h * LABEL_MAXS, y_v * LABEL_MAXS))
    mse_bl = np.square(np.subtract(y_v * LABEL_MAXS, y_b * LABEL_MAXS))
    row = np.zeros((len(y_h), 9))
    row[:, 8] = y_e
    row[:, 0:4] = mse_knn
    row[:, 4:8] = mse_bl
    out_table.append(row)


In [ ]:
x = np.concatenate(out_table, 0)
df = pd.DataFrame(
    x,
    columns=["P2O5", "K", "Mg", "pH", "P2O5_avg", "K_avg", "Mg_avg", "pH_avg", "Edge"],
)
df.head(10)
_, bin_edge = np.histogram(df.Edge.values, bins=4)
# print(bin_edge)
bin_edge = [0, 11, 40, 50, 100, 110, 120, 130, 210]
bin_edge_labels = [
    "0-11",
    "11-40",
    "40-50",
    "50-100",
    "100-110",
    "110-120",
    "120-130",
    "130+",
]
mse_per_edge = np.zeros((len(bin_edge) - 1, 6), dtype=object)

for i in range(1, len(bin_edge)):
    d_temp = df[(df.Edge <= bin_edge[i]) & (df.Edge > bin_edge[i - 1])]
    mse_per_edge[i - 1, 0] = np.mean(d_temp.P2O5.values) / np.mean(
        d_temp.P2O5_avg.values
    )
    mse_per_edge[i - 1, 1] = np.mean(d_temp.K.values) / np.mean(d_temp.K_avg.values)
    mse_per_edge[i - 1, 2] = np.mean(d_temp.Mg.values) / np.mean(d_temp.Mg_avg.values)
    mse_per_edge[i - 1, 3] = np.mean(d_temp.pH.values) / np.mean(d_temp.pH_avg.values)
    mse_per_edge[i - 1, 5] = len(d_temp)
    mse_per_edge[i - 1, 4] = bin_edge_labels[i - 1]



/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In the following cell, the mean validation score is demonstrated for each soil parameter and field size:

In [ ]:
d_out = pd.DataFrame(mse_per_edge, columns=["P2O5", "K", "Mg", "pH", "Edge", "Len"])
d_out

,P2O5,K,Mg,pH,Edge,Len
0,1.01189,0.957566,0.996239,0.710062,0-11,650
1,NaN,NaN,NaN,NaN,11-40,0
2,NaN,NaN,NaN,NaN,40-50,0
3,NaN,NaN,NaN,NaN,50-100,0
4,NaN,NaN,NaN,NaN,100-110,0
5,NaN,NaN,NaN,NaN,110-120,0
6,NaN,NaN,NaN,NaN,120-130,0
7,NaN,NaN,NaN,NaN,130+,0


In the following cell the mean validation score is calculated for each soil parameter.

#### 5.C.v. MAKE SUBMISSIONS ( FOR THE SMALL FIELDS <= 11x11 PIXELS)


In [ ]:
#Generate the submission for the small fields

predictions_small = []
counter = 0
for knn in KNN_models:
    pp = knn.predict(X_te_processed_normalized[avg_edge_test<=11, :])
    predictions_small.append(pp)

predictions_small = np.asarray(predictions_small)


predictions_small = np.mean(predictions_small, axis=0)
predictions_small = predictions_small * LABEL_MAXS # predictions were in the range of [0,1] so that we neeed to scale them back.

submission_df = pd.DataFrame(data=predictions_small, columns=["P", "K", "Mg", "pH"])
submission_df.to_csv("submission_small_fields.csv", index_label="sample_index")
submission_df

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7effcc2051f0>
Traceback (most recent call last):
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _Thread

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7eff1ca278b0>
Traceback (most recent call last):
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _Thread

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7eff1c9f49d0>
Traceback (most recent call last):
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/ai4ex2022/anaconda3/envs/ai4ex/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _Thread

,P,K,Mg,pH
0,69.046753,217.210041,156.625057,6.876837
1,68.314687,219.122053,159.695079,6.869646
2,68.406225,218.932231,159.838505,6.868999
3,70.056213,219.971472,160.081745,6.875113
4,70.007512,219.658213,165.494298,6.915121
...,...,...,...,...
427,70.749886,219.476535,165.619024,6.918288
428,69.396729,216.954871,157.758990,6.869399
429,69.869062,219.777196,159.857493,6.877047
430,72.610002,225.916411,163.957139,6.919657


### 5.D. MERGE SMALL AND LARGE FIELD SUBMISSIONS ( FINAL SUBMISSION FILE)

The final submission is the combined result of both models: The Random Forest for large patches and the KNN for small patches (11x11 pixels).

In [ ]:
submission_df = pd.DataFrame(data=np.concatenate([predictions_small , predictions_large],axis=0), columns=["P", "K", "Mg", "pH"])
submission_df.to_csv("submission_hybrid.csv", index_label="sample_index")
submission_df

,P,K,Mg,pH
0,69.046753,217.210041,156.625057,6.876837
1,68.314687,219.122053,159.695079,6.869646
2,68.406225,218.932231,159.838505,6.868999
3,70.056213,219.971472,160.081745,6.875113
4,70.007512,219.658213,165.494298,6.915121
...,...,...,...,...
1149,46.936877,169.610192,135.352855,6.628018
1150,45.545826,169.880842,135.613945,6.610217
1151,76.024305,239.529187,167.534027,6.643992
1152,47.691755,182.878088,138.305905,6.612771


In [ ]:
divisor = np.full(submission_df.shape ,np.array([ 70.30265589, 227.98851039, 159.28123557,   6.7827194 ]))

In [ ]:
sub = (submission_df / divisor).unstack().reset_index().rename(columns={0: 'Target'})
sub['sample_index'] = sub['level_1'].astype('str') + '_' + sub['level_0']
sub = sub[['sample_index', 'Target']]
sub

,sample_index,Target
0,0_P,0.982136
1,1_P,0.971723
2,2_P,0.973025
3,3_P,0.996495
4,4_P,0.995802
...,...,...
4611,1149_pH,0.977192
4612,1150_pH,0.974567
4613,1151_pH,0.979547
4614,1152_pH,0.974944


In [ ]:
sub.to_csv('baseline.csv', index=False)